# RAG With llama-index  + Milvus + LLama

References
- https://docs.llamaindex.ai/en/stable/examples/vector_stores/MilvusIndexDemo/
- https://docs.llamaindex.ai/en/stable/api_reference/storage/vector_store/milvus/?h=milvusvectorstore#llama_index.vector_stores.milvus.MilvusVectorStore

## Configuration

In [1]:
class MyConfig:
    pass

MY_CONFIG = MyConfig()

MY_CONFIG.EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
MY_CONFIG.EMBEDDING_LENGTH = 384

MY_CONFIG.INPUT_DATA_DIR = "input_data/walmart-reports-1"

MY_CONFIG.DB_URI = './rag_2_llamaindex.db'
MY_CONFIG.COLLECTION_NAME = 'llamaindex_walmart_docs'
MY_CONFIG.LLM_MODEL = "meta/meta-llama-3-8b-instruct"


In [2]:
%%time 

from llama_index.core import SimpleDirectoryReader
import pprint

# load documents
documents = SimpleDirectoryReader(
    # input_dir = './data/10k/input/'
    input_dir = MY_CONFIG.INPUT_DATA_DIR
).load_data()

print (f"Loaded {len(documents)} chunks")

# print("Document [0].doc_id:", documents[0].doc_id)
# pprint.pprint (documents[0], indent=4)

Loaded 300 chunks
CPU times: user 7.97 s, sys: 1.55 s, total: 9.52 s
Wall time: 7.7 s


In [3]:
# If connection to https://huggingface.co/ failed, uncomment the following path
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name = MY_CONFIG.EMBEDDING_MODEL
)

/home/sujee/my-stuff/projects/ai-alliance/data-prep-kit-sujee/examples/notebooks/rag/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(MY_CONFIG.DB_URI)
print ("✅ Connected to Milvus instance: ", MY_CONFIG.DB_URI )

# if we already have a collection, clear it first
if milvus_client.has_collection(collection_name = MY_CONFIG.COLLECTION_NAME):
    milvus_client.drop_collection(collection_name = MY_CONFIG.COLLECTION_NAME)
    print ('✅ Cleared collection :', MY_CONFIG.COLLECTION_NAME)
    

✅ Connected to Milvus instance:  ./rag_2_llamaindex.db
✅ Cleared collection : llamaindex_walmart_docs


In [6]:
# connect to vector db
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

vector_store = MilvusVectorStore(
    uri = MY_CONFIG.DB_URI ,
    dim = MY_CONFIG.EMBEDDING_LENGTH , 
    collection_name = MY_CONFIG.COLLECTION_NAME,
    overwrite=True
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print ("✅ Connected Llama-index to Milvus instance: ", MY_CONFIG.DB_URI )

✅ Connected Llama-index to Milvus instance:  ./rag_2_llamaindex.db


In [7]:
%%time

# create an index

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)
print ("✅ Created index:", index )

✅ Created index: <llama_index.core.indices.vector_store.base.VectorStoreIndex object at 0x74d7ff48b4d0>
CPU times: user 4.39 s, sys: 243 ms, total: 4.64 s
Wall time: 4 s


In [8]:
from llama_index.llms.replicate import Replicate
from llama_index.core import Settings

llm = Replicate(
    model= MY_CONFIG.LLM_MODEL,
    temperature=0.1
)

Settings.llm = llm

In [9]:
query_engine = index.as_query_engine()
res = query_engine.query("What was Walmart's revenue in 2023?")
print(res)



According to the provided context information, Walmart's total revenue in 2023 was $611,289 million.


In [10]:
query_engine = index.as_query_engine()
res = query_engine.query("How many distribution facilities does Walmart have?")
print(res)



Based on the provided context information, the answer to the query is:

Walmart has a total of 163 distribution facilities.


In [11]:
query_engine = index.as_query_engine()
res = query_engine.query("When was the moon landing?")
print(res)



I'm happy to help! However, I don't see any information about the moon landing in the provided context. The context appears to be a 10-K report filed by Walmart Inc. with the Securities and Exchange Commission. There is no mention of the moon landing in this report. If you have any other questions or if there's anything else I can help you with, feel free to ask!
